In [1]:
import requests
import pandas as pd
import time
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs

### 상장회사 전체 종목코드 크롤링

In [2]:
# 상장회사 전체 종목코드 들고오기

url = 'https://esg.krx.co.kr/contents/99/ESG99000001.jspx?code=/COM/finder_esg_company'

headers = {
    'Referer' : 'https://esg.krx.co.kr/templets/mobile/search-company.jsp?path=/contents/02/02010000/ESG02010000.jsp',
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

In [3]:
# 종목 정보가 없을 때 까지 크롤링 진행
stock_li, con_num = [], 1
while(True) :

    # josn 파일 api를 통해 가져오기
    j = requests.get(url, headers = headers, params = {'consonant' : con_num}).json()['block1']

    if j :      # CASE 1 : 종목 정보가 있는 경우
        stock_li += j
        con_num += 1

    else : break        # CASE 2 : 종목 정보가 없는 경우

# 모아진 종목 코드 데이터프레임으로 만들기
df_stock = pd.DataFrame(stock_li)
df_stock.columns = ['code', 'name']
df_stock

,code,name
0,000500,가온전선
1,000860,강남제비스코
2,035250,강원랜드
3,011420,갤럭시아에스엠
4,002100,경농
...,...,...
838,002820,SUN&L
839,002710,TCC스틸
840,002900,TYM
841,024070,WISCOM


### 각 회사(종목)별 ESG 뉴스 가져오기

In [4]:
# 각 종목별로 뉴스 가져오기
url = 'https://esg.krx.co.kr/contents/99/ESG99000001.jspx?code=/COM/finder_esg_news'

news_infos = []
for code in tqdm(df_stock['code']) :

    headers['Referer'] = f'https://esg.krx.co.kr/templets/mobile/news-list.jsp?isu_cd={code}'

    news_infos +=  requests.get(url, headers = headers, params = {'news_isu_cd' : code}).json()['block1']

  0%|          | 0/843 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
len(news_infos)

19228

In [5]:
news_infos

[{'isu_cd': '000500',
  'news_dd': '2023-07-12',
  'news_clss_nm': '사회',
  'lvl_val': '매우 높음',
  'title': '담합 온상 가온전선, 국세청 세무조사 배경에 촉각',
  'url': 'https://www.ajunews.com/view/20230712135619214'},
 {'isu_cd': '035250',
  'news_dd': '2023-09-18',
  'news_clss_nm': '지배구조',
  'lvl_val': '높음',
  'title': '강원랜드 부정 채용 피해자들 손배소 일부 승소',
  'url': 'http://www.kado.net/news/articleView.html?idxno=1204318'},
 {'isu_cd': '035250',
  'news_dd': '2023-09-17',
  'news_clss_nm': '지배구조',
  'lvl_val': '높음',
  'title': '강원랜드 채용 비리로 떨어진 지원자들, 손해배상 소송 승소',
  'url': 'https://www.kwnews.co.kr/page/view/2023091715240690772'},
 {'isu_cd': '035250',
  'news_dd': '2023-09-15',
  'news_clss_nm': '지배구조',
  'lvl_val': '매우 높음',
  'title': '강원랜드 채용비리...부정 청탁자는 해고, 일반 응시자는 배상',
  'url': 'https://www.ytn.co.kr/_ln/0115_202309151520462659'},
 {'isu_cd': '035250',
  'news_dd': '2023-09-15',
  'news_clss_nm': '지배구조',
  'lvl_val': '높음',
  'title': '강원랜드 채용 비리 탈락 피해자 21명, 손해배상 소송 승소',
  'url': 'http://www.kado.net/news/artic

### ESG 보고서 크롤링

In [ ]:
## ESG 보고서 크롤링

url = 'https://esg.krx.co.kr/contents/99/ESG99000001.jspx'

headers['Referer'] = 'https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp'

params = {
    'sch_com_nm' : "",
    'sch_yy' : 2023,
    'sch_tp' : 'N',
    'pagePath' : '/contents/02/02030000/ESG02030000.jsp',
    'code' : '02/02030000/esg02030000_01',
    'pageFirstCall' : 'Y',
    'curPage' : 1
}

company_li = []
while(True) :

    company_tem = requests.get(url, headers = headers, params = params).json()['result']

    if company_tem : company_li += company_tem
    else : break

    params['curPage'] += 1

print(f'보고서 존재 회사 : {len(company_li)}개')

보고서 존재 회사 : 153개


In [ ]:
headers['Referer'] = 'https://esg.krx.co.kr/'
parmas = {
    'method' : 'searchContents'
}

report_list, cnt = [], 1
for li in tqdm(company_li) :
    u = f'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={li["save_file_nm"]}&docno=&viewerhost=&viewerport='

    r = bs(requests.get(u, headers = headers).text, 'html.parser')

    doc_no = r.find('select', {'id' : 'attachedDoc'}).find_all('option')[-1]['value']
    u_ = f'https://kind.krx.co.kr/common/disclsviewer.do?method=searchContents&docNo={doc_no}'

    params['docNo'] = doc_no
    s = bs(requests.get(u_, params = params).text, 'html.parser')

    for u_report in s.find_all('script', {'type' : 'text/javascript'})[-1].text.split("'") :
        if u_report.startswith('https://kind') : break

    report_list.append(u_report[:-9] + bs(requests.get(u_report).text, 'html.parser').find_all("a")[0]["href"])

    if cnt % 5 : cnt += 1
    else :
        time.sleep(2)
        cnt += 1


  0%|          | 0/153 [00:00<?, ?it/s]

In [ ]:
df_report = pd.DataFrame(([x['com_abbrv'] for x in company_li], report_list), index = ['company', 'url']).T
df_report.to_csv('report_urls.csv', encoding = 'cp949', index = False)
df_report

,company,url
0,락앤락,https://kind.krx.co.kr/external/2023/10/11/000...
1,NH투자증권,https://kind.krx.co.kr/external/2023/09/20/000...
2,한국가스공사,https://kind.krx.co.kr/external/2023/09/12/000...
3,삼성증권,https://kind.krx.co.kr/external/2023/09/11/000...
4,DL,https://kind.krx.co.kr/external/2023/09/08/000...
...,...,...
148,카카오,https://kind.krx.co.kr/external/2023/05/30/000...
149,에스엘,https://kind.krx.co.kr/external/2023/05/31/000...
150,OCI홀딩스,https://kind.krx.co.kr/external/2023/05/26/000...
151,깨끗한나라,https://kind.krx.co.kr/external/2023/05/19/000...
